<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8cb77bb1850c5258351d1b8a8abb5883b1d289c3d528c80e3c329b8166a0115a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-21 14:52:45
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.35 C
-------------------
Today PnL: -79.31 K (-0.58%)
Current PnL: -33.90 L (-21.62%)
CY Booked + Current PnL: -19.61 L (-12.5%)
-------------------
Total profit:  1.15 L
Total loss:  -35.04 L
-------------------
Total Booked + Current PnL: 7.37 L (5.68%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.57%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 99.25 L (73.42%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-1.00,13.09,7.34,21.40,22033.0,34749.0,300175.0,4.93,47.0,X-LC,7.09,14.0,1.58,2.22,20.25,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-1.76,-24.67,32.87,0.09,24953.0,-24863.0,75914.0,76.09,33.0,M-SC,5.04,253.0,-1.00,0.56,0.00,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.04,2.16,16.70,19.23,25671.0,3256.0,153716.0,19.88,52.0,L-LC,10.52,182.0,0.13,1.14,23.90,XY25,ATH,MINING
49,MASFIN,398.61,-0.36,-7.36,31.75,22.05,28818.0,-7215.0,90765.0,-20.05,39.0,H-SC,4.73,168.0,-0.25,0.67,31.93,XR,ATH,FINANCE
32,HCLTECH,1908.19,-0.31,9.16,13.45,23.83,35518.0,22150.0,264074.0,14.03,59.0,X-LC,6.34,13.0,0.62,1.95,26.70,X40,ATH,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,5.48,-2.92,112.94,106.72,135225.0,-3604.0,119732.0,-51.96,55.0,H-SC,8.61,138.0,-0.03,0.89,19.77,AR,ATH,MISC
4,ALKYLAMINE,4546.37,3.06,-32.11,185.22,93.63,126953.0,-32421.0,68542.0,-27.23,48.0,H-SC,16.51,143.0,-0.26,0.51,3.62,SR,ATH,CHEMICALS
26,GILLETTE,11206.78,2.32,-4.58,38.77,32.42,93932.0,-11616.0,242280.0,-21.62,48.0,X-SC,12.24,87.0,-0.12,1.79,9.83,X40,ATH,FMCG
20,DABUR,735.00,2.20,0.92,42.39,43.70,106126.0,2289.0,250357.0,-7.41,55.0,X-MC,2.37,73.0,0.02,1.85,16.11,XY24,BTT,FMCG
83,WHIRLPOOL,2270.00,1.47,-36.20,185.14,81.92,135598.0,-41557.0,73241.0,-62.69,25.0,M-SC,29.26,218.0,-0.31,0.54,1.47,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.0,-4.21,-41.01,69.49,-0.01,52394.0,-52407.0,75398.0,15334.78,29.0,M-SC,14.06,235.0,-1.00,0.56,0.00,OX40N,BTT,AC
56,RAJOOENG,143.1,-3.96,-34.74,113.93,39.61,76208.0,-35610.0,66890.0,-55.84,48.0,H-SC,28.76,136.0,-0.47,0.49,8.27,AR,ATH,MISC
24,EASEMYTRIP,26.4,-3.59,-64.40,309.30,45.70,194670.0,-113876.0,62939.0,-34.60,17.0,M-SC,20.32,207.0,-0.58,0.47,0.00,XY24,NTT,TRAVEL
50,MEDANTA,1486.0,-3.46,-10.18,38.14,24.08,43489.0,-12926.0,114024.0,-17.52,30.0,X-SC,6.44,91.0,-0.30,0.84,5.74,XY24,NTT,HEALTHCARE
39,INDIGOPNTS,1408.0,-3.09,-21.55,27.49,0.02,37646.0,-37613.0,136946.0,108.71,36.0,M-SC,8.76,240.0,-1.00,1.01,19.12,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.11,-9.95,130.88,107.91,174014.0,-14687.0,132957.0,-26.63,32.0,M-SC,10.02,220.0,-0.08,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.09,-21.55,27.49,0.02,37646.0,-37613.0,136946.0,108.71,36.0,M-SC,8.76,240.0,-1.00,1.01,19.12,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-1.19,-23.29,72.10,32.03,72920.0,-30698.0,101137.0,-17.04,46.0,H-SC,6.44,157.0,-0.42,0.75,13.28,OX40N,NTT,IT
47,KANSAINER,340.00,-0.24,-24.47,50.24,13.47,102328.0,-65988.0,203679.0,-68.90,41.0,H-SC,4.37,159.0,-0.64,1.51,3.56,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.76,-24.67,32.87,0.09,24953.0,-24863.0,75914.0,76.09,33.0,M-SC,5.04,253.0,-1.00,0.56,0.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,0.04,2.16,16.7,19.23,25671.0,3256.0,153716.0,19.88,52.0,L-LC,10.52,182.0,0.13,1.14,23.9,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.13,-1.73,75.29,72.27,125374.0,-2926.0,166522.0,-15.61,32.0,M-LC,4.53,101.0,-0.02,1.23,4.34,XR,NTT,IT
38,INDIAMART,4810.62,5.48,-2.92,112.94,106.72,135225.0,-3604.0,119732.0,-51.96,55.0,H-SC,8.61,138.0,-0.03,0.89,19.77,AR,ATH,MISC
25,FINCABLES,1641.55,-2.11,-9.95,130.88,107.91,174014.0,-14687.0,132957.0,-26.63,32.0,M-SC,10.02,220.0,-0.08,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.18,-7.73,47.00,35.63,90881.0,-16210.0,193364.0,-20.73,35.0,H-MC,4.73,119.0,-0.18,1.43,7.65,AR,ATH,PHARMA
37,IEX,219.00,-1.24,-12.67,70.19,48.63,122565.0,-25335.0,174619.0,-41.93,36.0,H-SC,13.57,137.0,-0.21,1.29,0.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.01,-32.61,133.26,57.19,292867.0,-106356.0,219771.0,-68.10,15.0,X-MC,12.95,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
43,ITC,452.00,-0.43,-20.37,39.12,10.78,74862.0,-48952.0,191366.0,-52.60,16.0,X-LC,12.79,1.0,-0.65,1.42,0.0,X40,NTT,FMCG
58,RELAXO,1176.00,-1.69,-53.54,227.85,52.31,153653.0,-77724.0,67436.0,-50.54,16.0,X-SC,9.20,92.0,-0.51,0.50,0.0,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.64,-43.07,137.49,35.20,100714.0,-55418.0,73252.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.80,-10.95,50.69,34.19,96384.0,-23376.0,190144.0,-37.34,20.0,X-MC,5.46,60.0,-0.24,1.41,0.0,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,0.34,-14.41,38.51,18.55,124669.0,-54506.0,323731.0,-26.21,44.0,X-LC,0.97,3.0,-0.44,2.40,10.14,X40,ATH,IT
35,ICICIGI,2252.93,-1.41,-2.77,22.74,19.34,44244.0,-5546.0,194563.0,-23.33,31.0,X-MC,2.17,71.0,-0.13,1.44,12.72,X40,ATH,INSURANCE
33,HINDUNILVR,2922.00,-0.26,-7.69,23.14,13.67,54909.0,-19773.0,237290.0,-14.61,52.0,X-LC,2.29,9.0,-0.36,1.76,12.02,XY25,NTT,FMCG
20,DABUR,735.00,2.20,0.92,42.39,43.70,106126.0,2289.0,250357.0,-7.41,55.0,X-MC,2.37,73.0,0.02,1.85,16.11,XY24,BTT,FMCG
78,UNITDSPR,1644.00,0.02,-3.11,24.65,20.77,55919.0,-7293.0,226851.0,-11.53,34.0,X-MC,2.48,70.0,-0.13,1.68,3.59,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.01,-32.61,133.26,57.19,292867.0,-106356.0,219771.0,-68.10,15.0,X-MC,12.95,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.51,-23.49,103.20,55.47,192100.0,-57143.0,186143.0,-32.12,23.0,X-MC,15.98,63.0,-0.30,1.38,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.64,-43.07,137.49,35.20,100714.0,-55418.0,73252.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.34,-26.93,79.63,31.26,134275.0,-62144.0,168624.0,-65.86,25.0,X-MC,24.00,56.0,-0.46,1.25,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-2.25,-15.74,57.23,32.48,158161.0,-51631.0,276360.0,-20.01,28.0,X-MC,5.28,69.0,-0.33,2.04,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.18,-33.82,115.85,42.84,49836.0,-21988.0,43018.0,-56.47,32.0,X-SC,23.04,83.0,-0.44,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-1.69,-53.54,227.85,52.31,153653.0,-77724.0,67436.0,-50.54,16.0,X-SC,9.20,92.0,-0.51,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.64,-43.07,137.49,35.20,100714.0,-55418.0,73252.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.46,-10.18,38.14,24.08,43489.0,-12926.0,114024.0,-17.52,30.0,X-SC,6.44,91.0,-0.30,0.84,5.74,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.18,-17.81,74.38,43.32,99565.0,-29012.0,133860.0,-29.42,45.0,X-SC,8.18,90.0,-0.29,0.99,2.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.43,-20.37,39.12,10.78,74862.0,-48952.0,191366.0,-52.60,16.0,X-LC,12.79,1.0,-0.65,1.42,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.79,-30.75,77.59,22.98,121370.0,-69473.0,156425.0,-27.96,31.0,X-LC,3.62,2.0,-0.57,1.16,0.00,XY24,NTT,AUTO
73,TCS,4311.59,0.34,-14.41,38.51,18.55,124669.0,-54506.0,323731.0,-26.21,44.0,X-LC,0.97,3.0,-0.44,2.40,10.14,X40,ATH,IT
81,VBL,671.64,-2.58,-3.75,40.95,35.67,124491.0,-11835.0,304007.0,-15.14,41.0,X-LC,2.88,5.0,-0.10,2.25,9.59,X40N,ATH,FMCG
41,INFY,1972.00,-0.54,12.32,19.52,34.24,68603.0,38562.0,351450.0,-12.42,56.0,X-LC,7.27,6.0,0.56,2.60,21.63,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.35,-32.94,98.16,32.89,50823.0,-25430.0,51776.0,-739.67,55.0,L-MC,10.44,259.0,-0.50,0.38,42.59,XR,NTT,BANKS
17,COALINDIA,484.83,0.04,2.16,16.70,19.23,25671.0,3256.0,153716.0,19.88,52.0,L-LC,10.52,182.0,0.13,1.14,23.90,XY25,ATH,MINING
5,ASIANPAINT,3460.25,-0.61,-7.05,30.12,20.95,70484.0,-17758.0,234010.0,-7.94,35.0,X-LC,12.60,36.0,-0.25,1.73,26.07,X40,ATH,PAINTS
32,HCLTECH,1908.19,-0.31,9.16,13.45,23.83,35518.0,22150.0,264074.0,14.03,59.0,X-LC,6.34,13.0,0.62,1.95,26.70,X40,ATH,IT
51,NESTLEIND,1377.00,-1.00,13.09,7.34,21.40,22033.0,34749.0,300175.0,4.93,47.0,X-LC,7.09,14.0,1.58,2.22,20.25,XY25,NTT,FMCG


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.84,-32.41,67.04,12.90,99005.0,-70816.0,147680.0,117.55,50.0,H-SC,7.44,173.0,-0.72,1.09,22.79,OX40N,NTT,FINANCE
17,COALINDIA,484.83,0.04,2.16,16.70,19.23,25671.0,3256.0,153716.0,19.88,52.0,L-LC,10.52,182.0,0.13,1.14,23.90,XY25,ATH,MINING
62,SATIN,274.00,0.71,-17.01,82.95,51.83,190203.0,-46986.0,229298.0,-25.10,57.0,H-SC,3.31,148.0,-0.25,1.70,11.70,XY24,NTT,FINANCE
40,INDUSINDBK,1800.00,0.35,-32.94,98.16,32.89,50823.0,-25430.0,51776.0,-739.67,55.0,L-MC,10.44,259.0,-0.50,0.38,42.59,XR,NTT,BANKS
20,DABUR,735.00,2.20,0.92,42.39,43.70,106126.0,2289.0,250357.0,-7.41,55.0,X-MC,2.37,73.0,0.02,1.85,16.11,XY24,BTT,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.62
1,25,44.95
2,50,76.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.42
MC    30.70
LC    26.90
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.30
X40      24.23
X40N     12.88
XY25     10.91
XR        8.73
AR        8.48
OX40N     7.53
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.29
X-LC    23.47
H-SC    22.72
M-SC    10.98
X-SC     7.95
H-MC     4.58
M-MC     1.45
M-LC     1.23
L-LC     1.14
H-LC     1.06
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.53,-11.22,47.71
IT,12.86,-23.00,84.44
FINANCE,11.06,-19.30,69.38
MISC,7.06,-36.17,91.87
PAINTS,5.79,-17.93,35.45
ELECTRICAL,5.76,-20.44,64.56
INSURANCE,4.75,-5.70,41.87
PHARMA,3.99,-8.98,43.28
BANKS,2.78,-42.13,128.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3417777.0,21
AR,1425799.0,10
XR,1310674.0,12
X40,1234040.0,15
X40N,1003087.0,9
OX40N,772528.0,10
XY25,469826.0,7
SR,291138.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3723708.0,24
X-MC,1750978.0,16
M-SC,1574392.0,15
X-LC,1043540.0,13
X-SC,843574.0,8
H-MC,460183.0,3
L-SC,186990.0,2
M-LC,125374.0,1
H-LC,89747.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1303348.0      6
           AR         951656.0      5
M-SC       XY24       894052.0      6
H-SC       XR         848954.0      7
X-MC       X40        599946.0      7
           XY24       466090.0      3
X-LC       X40        439448.0      6
X-MC       X40N       438919.0      4
M-SC       OX40N      348781.0      5
X-SC       X40N       335949.0      3
H-SC       OX40N      328612.0      4
X-SC       XY24       312979.0      3
H-SC       SR         291138.0      2
H-MC       AR         246505.0      2
X-MC       XY25       246023.0      2
X-LC       X40N       228219.0      2
           XY24       227630.0      2
H-MC       XY24       213678.0      1
X-SC       X40        194646.0      2
M-SC       XR         193668.0      2
X-LC       XY25       148243.0      3
M-SC       AR         137891.0      2
M-LC       XR         125374.0      1
L-SC       OX40N       95135.0      1
           XR          91855.0      1
H-LC       AR          89747.0      1
L-MC       XR          50823.0      1
M-MC       XY25        49889.0      1
L-LC       XY25        25671.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
